## API Time Selectors - Using Start, End, and Reftime with Forecast Data

Forecast datasets typically have two time dimensions, `reftime` and `time`. The `reftime` of a forecast data product represents the run time of the forecast model, while `time` represents the actual time (typically in the future) that the values are valid. For each `reftime`, values are unique for each unique `time` value.

Typically the most recent forecast is most desirable to users, and the Planet OS API intentionally returns these values by default. However, there may be cases when one wants to investigate a historical forecast, or how values at a specific time differ between successive forecasts.

The [RTOFS data product](http://data.planetos.com/datasets/noaa_rtofs_surface_1h_diag) is a daily updated forecast that provides a 3-day, hourly forecast. This notebook highlights ways to use the start, end, reftime_start, and reftime_end API parameters and explain the expected response.

In [1]:
%matplotlib inline

import pandas as pd
import simplejson as json
from urllib.parse import urlencode
from urllib.request import urlopen, Request
import datetime
import numpy as np

A Planet OS API key is required to run this notebook. Keys are displayed in the [account settings](http://data.planetos.com/account/settings/) page on the Planet OS Datahub. If you do not have a Planet OS account, you can [sign up for free](http://data.planetos.com/plans).

In [2]:
apikey = open('APIKEY').readlines()[0].strip() #'<YOUR API KEY HERE>'

## Request the Most Recent Model Run

By default, our API returns the most recent model run. The example below returns all variables for the most recent forecast run (e.g. reftime). At the time of running, this was 2016-11-12T00:00:00. Given the hourly resolution and 3-day extent, we expect 72 unique values to be returned.

In [3]:
# Set the Planet OS API query parameters
count = 100
id = 'noaa_rtofs_surface_1h_diag'
lat=39.37858604638528
lon=-72.57739563685647
time_order = 'desc'

query_dict = {'apikey': apikey,
              'count': count,
              'lon': lon,
              'lat': lat,
              'time_order': time_order,
              }
query = urlencode(query_dict)

api_url = "http://api.planetos.com/v1/datasets/%s/point?%s" % (id, query)

request = Request(api_url)
response = urlopen(request)
response_json = json.loads(response.read())

data = response_json['entries']

# let's flatten the response and create a Pandas dataframe
df = pd.io.json.json_normalize(data)

# then index by time using the axes.time column
pd.to_datetime(df["axes.time"])
df.set_index('axes.time', inplace=False)

print(df.count())
df.head()

axes.latitude                            73
axes.longitude                           73
axes.reftime                             73
axes.time                                73
classifiers.reference_time               73
context                                  73
data.ice_coverage                        73
data.ice_thickness                       73
data.mixed_layer_thickness               73
data.ssh                                 73
data.surface_boundary_layer_thickness    73
data.u_barotropic_velocity               73
data.v_barotropic_velocity               73
dtype: int64


,axes.latitude,axes.longitude,axes.reftime,axes.time,classifiers.reference_time,context,data.ice_coverage,data.ice_thickness,data.mixed_layer_thickness,data.ssh,data.surface_boundary_layer_thickness,data.u_barotropic_velocity,data.v_barotropic_velocity
0,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-18T00:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,2.393272,-0.586881,1.003857,0.052923,-0.051910
1,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T23:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,1.461914,-0.582981,1.005212,0.048648,-0.042568
2,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T22:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,0.939150,-0.577908,1.008560,0.047611,-0.033235
3,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T21:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,0.904634,-0.573326,1.012362,0.049494,-0.023227
4,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T20:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,0.972110,-0.571742,1.015378,0.052853,-0.012663


## Request a Single Model Run

By default, the system will return the most recent model run (e.g. reftime). However, a specific model run can be acquired by requesting an identical reftime_start and reftime_end value. In the example below, we request values from only the 2016-11-12T00:00:00 model run.

In [4]:
df

,axes.latitude,axes.longitude,axes.reftime,axes.time,classifiers.reference_time,context,data.ice_coverage,data.ice_thickness,data.mixed_layer_thickness,data.ssh,data.surface_boundary_layer_thickness,data.u_barotropic_velocity,data.v_barotropic_velocity
0,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-18T00:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,2.393272,-0.586881,1.003857,0.052923,-0.051910
1,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T23:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,1.461914,-0.582981,1.005212,0.048648,-0.042568
2,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T22:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,0.939150,-0.577908,1.008560,0.047611,-0.033235
3,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T21:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,0.904634,-0.573326,1.012362,0.049494,-0.023227
4,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T20:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,0.972110,-0.571742,1.015378,0.052853,-0.012663
5,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T19:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,1.099548,-0.573966,1.016540,0.056213,-0.002916
6,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T18:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,1.365773,-0.577964,1.015528,0.059150,0.003989
7,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T17:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,2.068107,-0.581208,1.012883,0.062220,0.006222
8,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T16:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,4.017891,-0.582298,1.009745,0.066123,0.003401
9,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-17T15:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,5.489046,-0.580531,1.007133,0.071520,-0.003411


In [5]:
# Request only data from a specific model run
reftime_end = np.max(df['axes.reftime'])
reftime_start = np.max(df['axes.reftime'])

query_dict = {'apikey': apikey,
              'count': count,
              'lat': lat,
              'lon': lon,
              'reftime_end': reftime_end,
              'reftime_start': reftime_start,
              'time_order': time_order,
              }
query = urlencode(query_dict)

api_url = "http://api.planetos.com/v1/datasets/%s/point?%s" % (id, query)

request = Request(api_url)
response = urlopen(request)
response_json = json.loads(response.read())

data = response_json['entries']

# let's flatten the response and create a Pandas dataframe
df = pd.io.json.json_normalize(data)

# then index by time using the axes.time column
pd.to_datetime(df["axes.time"])
df.set_index('axes.time', inplace=False)

print(df.count())

axes.latitude                            73
axes.longitude                           73
axes.reftime                             73
axes.time                                73
classifiers.reference_time               73
context                                  73
data.ice_coverage                        73
data.ice_thickness                       73
data.mixed_layer_thickness               73
data.ssh                                 73
data.surface_boundary_layer_thickness    73
data.u_barotropic_velocity               73
data.v_barotropic_velocity               73
dtype: int64


### All available model run values for a specific time

This example looks for data at a specific future datetime (e.g. 2016-11-14T00:00:00) and returns values from all available model runs (e.g. reftimes). From the result at the time of running, we can see that two forecasts provide values for Nov 14th at 00:00, the 2016-11-11T00:00:00 and 2016-11-12T00:00:00 model runs.

In [6]:
# Request only data from past model run (e.g. 2016-11-11T00:00:00)

end = np.min(df['axes.time'])
start = datetime.datetime.strftime(datetime.datetime.strptime(df['axes.reftime'][0],'%Y-%m-%dT%H:%M:%S') - datetime.timedelta(days=1),'%Y-%m-%dT%H:%M:%S')

query_dict = {'apikey': apikey,
              'count': count,
              'end': end,
              'lat': lat,
              'lon': lon,
              'reftime_recent': 'false',
              'start': start,
              'time_order': time_order,
              }
query = urlencode(query_dict)

api_url = "http://api.planetos.com/v1/datasets/%s/point?%s" % (id, query)

request = Request(api_url)
response = urlopen(request)
response_json = json.loads(response.read())

data = response_json['entries']

# let's flatten the response and create a Pandas dataframe
df = pd.io.json.json_normalize(data)
print (df)
# then index by time using the axes.time column
pd.to_datetime(df["axes.time"])
df.set_index('axes.time', inplace=True)

print(df.count())
df.head()

   axes.latitude  axes.longitude         axes.reftime            axes.time  \
0      39.390355      -72.616137  2018-06-15T00:00:00  2018-06-15T00:00:00   

  classifiers.reference_time                    context  data.ice_coverage  \
0   2018-06-15T00:00:00.000Z  reference_time_MT_lat_lon                0.0   

   data.ice_thickness  data.mixed_layer_thickness  data.ssh  \
0                 0.0                    6.393537 -0.574618   

   data.surface_boundary_layer_thickness  data.u_barotropic_velocity  \
0                               6.448367                    0.068051   

   data.v_barotropic_velocity  
0                   -0.051244  
axes.latitude                            1
axes.longitude                           1
axes.reftime                             1
classifiers.reference_time               1
context                                  1
data.ice_coverage                        1
data.ice_thickness                       1
data.mixed_layer_thickness               1
data.s

,axes.latitude,axes.longitude,axes.reftime,classifiers.reference_time,context,data.ice_coverage,data.ice_thickness,data.mixed_layer_thickness,data.ssh,data.surface_boundary_layer_thickness,data.u_barotropic_velocity,data.v_barotropic_velocity
axes.time,,,,,,,,,,,,
2018-06-15T00:00:00,39.390355,-72.616137,2018-06-15T00:00:00,2018-06-15T00:00:00.000Z,reference_time_MT_lat_lon,0.0,0.0,6.393537,-0.574618,6.448367,0.068051,-0.051244
